In [ ]:
# TODO scope diff between tradable price and global price to see if interesting to show in exploratory data analysis
# -show how far off you'd be by using this price as opposed to the actually available price
# -can note it'd be even worse for assets that i'm not looking as they'd be adversely selected toward lower liquidity


In [ ]:
# TODO CAN USE THIS FOR SCOPING UNIVARIATE CORR

series2 = df.price_usd[1:].values
for col in df.columns.values[1:-2]:
    series1 = df[col][:-1].values

    # Find indices where both series have non-missing values
    non_missing_indices = np.logical_not(np.isnan(series1) | np.isnan(series2))

    # Compute the correlation using non-missing values only
    corr_matrix = np.corrcoef(series1[non_missing_indices], series2[non_missing_indices])

    # Extract the correlation coefficient (off-diagonal element)
    corr_coef = corr_matrix[0, 1]
    print(col)
    print(f"Correlation coefficient: {corr_coef}")


In [ ]:
# TODO FOR DESC STAT + LOW DIM FM: ENSURE I CREATE ALL CHAR/FACTORS USED IN LIT
# -CREATE BOTH FOR HOUR OVER HOUR AND DAY OVER DAY

In [ ]:
# TODO feat eng for macro data:
# -form snp return
# -form snp momentum
# -form snp volatility
# -form squared market return
# -form all chen transformations too
# -CREATE BOTH FOR HOUR OVER HOUR AND DAY OVER DAY

In [ ]:
# TODO COMPRESS DOWN THE CHARACTERISTICS AND MACRO VARIABLES FOR REPEATED ONES
# -FOR ONES THAT ARE REALLY HARD TO DECIDE / POTENTIALLY IMPORTANT THEN CREATE THE UNI CORR TABLES TO DECIDE LATER (E.G. MCAP)
# -COMPRESS DOWN THE MACRO COVARIATES TO SOMETHING CLOSER TO GU 2019 INSTEAD OF RAW MCCRAKEN
# -CREATE BOTH FOR HOUR OVER HOUR AND DAY OVER DAY

In [ ]:
# TODO FOR THE DESC STAT AND LOW DIM FM STUFF, LETS WORK AT DAILY LEVEL
# -TO REDUCE OUT SOME OF THE NOISE
# -REDUCE THE SAMPLE SIZE FOR MORE SIMPLE MODELS TO WORK
# -HAVE RESULTS BE SUGGESTIVE OF HOURLY LEVEL PERFORMANCE


In [ ]:
# TODO FOR THE HIGH DIM FM PANEL AT HOURLY FREQ, 
# -LETS WORK WITH DATA IN PRETTY RAW FORM AS NEURAL NETS WILL LEARN HOW TO DO FEAT ENG
# -MAYBE SOME OF THE COLUMNS WE CAN SUMMARIZE IF THEIR COVARIANCE IS PRETTY USELESS

In [ ]:
# FORM RETURN VARIABLES

# Ensure it is sorted
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Form RHS return variables
assert(0==df[df.usd_per_token == 0].shape[0])
df['covar_r_t']               = df.groupby('asset')['usd_per_token'].pct_change(periods=7)
df['covar_r_tm14']            = df.groupby('asset')['usd_per_token'].pct_change(periods=14)
df['covar_r_tm21']            = df.groupby('asset')['usd_per_token'].pct_change(periods=21)
df['covar_r_tm28']            = df.groupby('asset')['usd_per_token'].pct_change(periods=28)
df['covar_r_daily_t']         = df.groupby('asset')['usd_per_token'].pct_change(periods=1)
df['covar_r_daily_avg_tm7']   = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(7).mean())
df['covar_vol_r_daily_tm7']   = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(7).std())
df['covar_vol_r_daily_tm14']  = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(14).std())
df['covar_vol_r_daily_tm21']  = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(21).std())
df['covar_vol_r_daily_tm28']  = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(28).std())
df['covar_skew_r_daily_tm7']  = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(7).skew())
df['covar_skew_r_daily_tm14'] = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(14).skew())
df['covar_skew_r_daily_tm21'] = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(21).skew())
df['covar_skew_r_daily_tm28'] = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(28).skew())
df['covar_kurt_r_daily_tm7']  = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(7).kurt())
df['covar_kurt_r_daily_tm14'] = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(14).kurt())
df['covar_kurt_r_daily_tm21'] = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(21).kurt())
df['covar_kurt_r_daily_tm28'] = df.groupby('asset')['covar_r_daily_t'].transform(lambda x: x.rolling(28).kurt())

# Form LHS variable
df['r_tplus7'] = df.groupby('asset')['covar_r_t'].shift(-7)

# Drop rows missing r_tplus7
df = df[~df.r_tplus7.isnull()]

# Drop 2022 rows
df = df[df.date <= '2021-12-26']

# Ensure it is sorted and reset index
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# FORM AGE VARIABLE

df['covar_age_t'] = (pd.to_datetime(df.date) - pd.to_datetime(df.first_date_cmc)).astype('timedelta64[D]')
assert(0 <= np.min(df.covar_age_t))
df = df.drop('first_date_cmc', axis=1)

# FORM PRICE VARIABLES

# Sunday price and log price
df['covar_p_t']     = df.usd_per_token.values
df['covar_p_log_t'] = np.log(df.usd_per_token.values)

# Avg, min, & max price and log price in last week and last 28 days
df['covar_p_avg_daily_tm7']      = df.groupby('asset')['usd_per_token'].transform(lambda x: x.rolling(7).mean())
df['covar_p_min_daily_tm7']      = df.groupby('asset')['usd_per_token'].transform(lambda x: x.rolling(7).min())
df['covar_p_max_daily_tm7']      = df.groupby('asset')['usd_per_token'].transform(lambda x: x.rolling(7).max())
df['covar_p_log_avg_daily_tm7']  = df.groupby('asset')['usd_per_token'].transform(lambda x: np.log(x).rolling(7).mean())
df['covar_p_log_min_daily_tm7']  = df.groupby('asset')['usd_per_token'].transform(lambda x: np.log(x).rolling(7).min())
df['covar_p_log_max_daily_tm7']  = df.groupby('asset')['usd_per_token'].transform(lambda x: np.log(x).rolling(7).max())
df['covar_p_avg_daily_tm28']     = df.groupby('asset')['usd_per_token'].transform(lambda x: x.rolling(28).mean())
df['covar_p_min_daily_tm28']     = df.groupby('asset')['usd_per_token'].transform(lambda x: x.rolling(28).min())
df['covar_p_max_daily_tm28']     = df.groupby('asset')['usd_per_token'].transform(lambda x: x.rolling(28).max())
df['covar_p_log_avg_daily_tm28'] = df.groupby('asset')['usd_per_token'].transform(lambda x: np.log(x).rolling(28).mean())
df['covar_p_log_min_daily_tm28'] = df.groupby('asset')['usd_per_token'].transform(lambda x: np.log(x).rolling(28).min())
df['covar_p_log_max_daily_tm28'] = df.groupby('asset')['usd_per_token'].transform(lambda x: np.log(x).rolling(28).max())

# FORM MCAP VARIABLES

# Sunday mcap and log mcap
df['covar_mcap_t']     = df.usd_mcap.values
df['covar_mcap_log_t'] = np.log(df.usd_mcap.values)

# Avg, min, & max mcap and log mcap in last week and last 28 days
df['covar_mcap_avg_daily_tm7']      = df.groupby('asset')['usd_mcap'].transform(lambda x: x.rolling(7).mean())
df['covar_mcap_min_daily_tm7']      = df.groupby('asset')['usd_mcap'].transform(lambda x: x.rolling(7).min())
df['covar_mcap_max_daily_tm7']      = df.groupby('asset')['usd_mcap'].transform(lambda x: x.rolling(7).max())
df['covar_mcap_log_avg_daily_tm7']  = df.groupby('asset')['usd_mcap'].transform(lambda x: np.log(x).rolling(7).mean())
df['covar_mcap_log_min_daily_tm7']  = df.groupby('asset')['usd_mcap'].transform(lambda x: np.log(x).rolling(7).min())
df['covar_mcap_log_max_daily_tm7']  = df.groupby('asset')['usd_mcap'].transform(lambda x: np.log(x).rolling(7).max())
df['covar_mcap_avg_daily_tm28']     = df.groupby('asset')['usd_mcap'].transform(lambda x: x.rolling(28).mean())
df['covar_mcap_min_daily_tm28']     = df.groupby('asset')['usd_mcap'].transform(lambda x: x.rolling(28).min())
df['covar_mcap_max_daily_tm28']     = df.groupby('asset')['usd_mcap'].transform(lambda x: x.rolling(28).max())
df['covar_mcap_log_avg_daily_tm28'] = df.groupby('asset')['usd_mcap'].transform(lambda x: np.log(x).rolling(28).mean())
df['covar_mcap_log_min_daily_tm28'] = df.groupby('asset')['usd_mcap'].transform(lambda x: np.log(x).rolling(28).min())
df['covar_mcap_log_max_daily_tm28'] = df.groupby('asset')['usd_mcap'].transform(lambda x: np.log(x).rolling(28).max())

# FORM VOLUME VARIABLES

# Sunday volume and log volume
df['covar_volume_t']     = df.usd_trading_volume_24h.values
df['covar_volume_log_t'] = np.log(df.usd_trading_volume_24h.values)

# Total, avg, median, min, & max volume and log volume over last week and last 28 days
df['covar_volume_sum_daily_tm7']      = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(7).sum())
df['covar_volume_avg_daily_tm7']      = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(7).mean())
df['covar_volume_med_daily_tm7']      = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(7).median())
df['covar_volume_min_daily_tm7']      = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(7).min())
df['covar_volume_max_daily_tm7']      = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(7).max())
df['covar_volume_log_sum_daily_tm7']  = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(7).sum())
df['covar_volume_log_avg_daily_tm7']  = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(7).mean())
df['covar_volume_log_med_daily_tm7']  = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(7).median())
df['covar_volume_log_min_daily_tm7']  = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(7).min())
df['covar_volume_log_max_daily_tm7']  = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(7).max())
df['covar_volume_sum_daily_tm28']     = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(28).sum())
df['covar_volume_avg_daily_tm28']     = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(28).mean())
df['covar_volume_med_daily_tm28']     = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(28).median())
df['covar_volume_min_daily_tm28']     = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(28).min())
df['covar_volume_max_daily_tm28']     = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: x.rolling(28).max())
df['covar_volume_log_sum_daily_tm28'] = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(28).sum())
df['covar_volume_log_avg_daily_tm28'] = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(28).mean())
df['covar_volume_log_med_daily_tm28'] = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(28).median())
df['covar_volume_log_min_daily_tm28'] = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(28).min())
df['covar_volume_log_max_daily_tm28'] = df.groupby('asset')['usd_trading_volume_24h'].transform(lambda x: np.log(x).rolling(28).max())
# FORM PRICE VOLUME VARIABLES

# Sunday price times volume and log of it
df['covar_p_volume_t']     = df.covar_p_t * df.covar_volume_t
df['covar_p_volume_log_t'] = df.covar_p_log_t * df.covar_volume_log_t

# Log average daily volumes times avg price over past week and 28 days. Std too
df['covar_p_volume_log_avg_daily_tm7']  = np.log(df.covar_volume_avg_daily_tm7 * df.covar_p_avg_daily_tm7)
df['covar_p_volume_log_avg_daily_tm28'] = np.log(df.covar_volume_avg_daily_tm28 * df.covar_p_avg_daily_tm28)
df['covar_p_volume_log_std_daily_tm7']  = np.log(df.groupby('asset')['covar_p_volume_t'].transform(lambda x: x.rolling(7).std()))
df['covar_p_volume_log_std_daily_tm28'] = np.log(df.groupby('asset')['covar_p_volume_t'].transform(lambda x: x.rolling(28).std()))

# DROP PRICE, MCAP, VOLUME VARS

df = df.drop(['usd_per_token', 'usd_mcap', 'usd_trading_volume_24h'], axis=1)

# Ensure it is sorted and reset index
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

In [ ]:
# FORM MACRO VARIABLES

# Confirm some columns are not missing for to be made pct change columns
assert(0==df[df['active_cryptocurrencies_cmc'].isnull()].shape[0])
assert(0==df[df['active_exchanges_cmc'].isnull()].shape[0])
assert(0==df[df['active_market_pairs_cmc'].isnull()].shape[0])
assert(0==df[df['btc_dominance_cmc'].isnull()].shape[0])
assert(0==df[(df.asset == 'ethereum') & df.covar_mcap_t.isnull()].shape[0])

# Trading volume variables
df['macro_volume_t']             = df.total_volume_24h_cmc.values
df['macro_volume_altcoin_t']     = df.altcoin_volume_24h_cmc.values
df['macro_volume_log_t']         = np.log(df.total_volume_24h_cmc.values)
df['macro_volume_altcoin_log_t'] = np.log(df.altcoin_volume_24h_cmc.values)
df['macro_volume_reported_t']             = df.total_volume_24h_reported_cmc.values
df['macro_volume_altcoin_reported_t']     = df.altcoin_volume_24h_reported_cmc.values
df['macro_volume_reported_log_t']         = np.log(df.total_volume_24h_reported_cmc.values)
df['macro_volume_altcoin_reported_log_t'] = np.log(df.altcoin_volume_24h_reported_cmc.values)
temp_df = df[df.asset == 'bitcoin'][['date', 'total_volume_24h_cmc', 'altcoin_volume_24h_cmc',
                                     'total_volume_24h_reported_cmc', 'altcoin_volume_24h_reported_cmc',
                                     'covar_volume_t', 'covar_volume_log_t']]
temp_df = temp_df.rename(columns={'covar_volume_t': 'macro_volume_btc_t',
                                  'covar_volume_log_t': 'macro_volume_btc_log_t'})
temp_df['macro_volume_sum_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(7).sum())
temp_df['macro_volume_avg_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(7).mean())
temp_df['macro_volume_min_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(7).min())
temp_df['macro_volume_max_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(7).max())
temp_df['macro_volume_reported_sum_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).sum())
temp_df['macro_volume_reported_avg_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).mean())
temp_df['macro_volume_reported_min_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).min())
temp_df['macro_volume_reported_max_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).max())
temp_df['macro_volume_log_sum_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).sum())
temp_df['macro_volume_log_avg_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).mean())
temp_df['macro_volume_log_min_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).min())
temp_df['macro_volume_log_max_daily_tm7'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).max())
temp_df['macro_volume_log_reported_sum_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).sum())
temp_df['macro_volume_log_reported_avg_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).mean())
temp_df['macro_volume_log_reported_min_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).min())
temp_df['macro_volume_log_reported_max_daily_tm7'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).max())
temp_df['macro_volume_sum_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(28).sum())
temp_df['macro_volume_avg_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(28).mean())
temp_df['macro_volume_min_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(28).min())
temp_df['macro_volume_max_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: x.rolling(28).max())
temp_df['macro_volume_reported_sum_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).sum())
temp_df['macro_volume_reported_avg_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).mean())
temp_df['macro_volume_reported_min_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).min())
temp_df['macro_volume_reported_max_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).max())
temp_df['macro_volume_log_sum_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).sum())
temp_df['macro_volume_log_avg_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).mean())
temp_df['macro_volume_log_min_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).min())
temp_df['macro_volume_log_max_daily_tm28'] = temp_df.total_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).max())
temp_df['macro_volume_log_reported_sum_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).sum())
temp_df['macro_volume_log_reported_avg_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).mean())
temp_df['macro_volume_log_reported_min_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).min())
temp_df['macro_volume_log_reported_max_daily_tm28'] = temp_df.total_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).max())
temp_df['macro_volume_altcoin_sum_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(7).sum())
temp_df['macro_volume_altcoin_avg_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(7).mean())
temp_df['macro_volume_altcoin_min_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(7).min())
temp_df['macro_volume_altcoin_max_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(7).max())
temp_df['macro_volume_altcoin_reported_sum_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).sum())
temp_df['macro_volume_altcoin_reported_avg_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).mean())
temp_df['macro_volume_altcoin_reported_min_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).min())
temp_df['macro_volume_altcoin_reported_max_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(7).max())
temp_df['macro_volume_altcoin_log_sum_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).sum())
temp_df['macro_volume_altcoin_log_avg_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).mean())
temp_df['macro_volume_altcoin_log_min_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).min())
temp_df['macro_volume_altcoin_log_max_daily_tm7'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(7).max())
temp_df['macro_volume_altcoin_log_reported_sum_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).sum())
temp_df['macro_volume_altcoin_log_reported_avg_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).mean())
temp_df['macro_volume_altcoin_log_reported_min_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).min())
temp_df['macro_volume_altcoin_log_reported_max_daily_tm7'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(7).max())
temp_df['macro_volume_altcoin_sum_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(28).sum())
temp_df['macro_volume_altcoin_avg_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(28).mean())
temp_df['macro_volume_altcoin_min_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(28).min())
temp_df['macro_volume_altcoin_max_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: x.rolling(28).max())
temp_df['macro_volume_altcoin_reported_sum_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).sum())
temp_df['macro_volume_altcoin_reported_avg_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).mean())
temp_df['macro_volume_altcoin_reported_min_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).min())
temp_df['macro_volume_altcoin_reported_max_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: x.rolling(28).max())
temp_df['macro_volume_altcoin_log_sum_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).sum())
temp_df['macro_volume_altcoin_log_avg_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).mean())
temp_df['macro_volume_altcoin_log_min_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).min())
temp_df['macro_volume_altcoin_log_max_daily_tm28'] = temp_df.altcoin_volume_24h_cmc.transform(lambda x: np.log(x).rolling(28).max())
temp_df['macro_volume_altcoin_log_reported_sum_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).sum())
temp_df['macro_volume_altcoin_log_reported_avg_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).mean())
temp_df['macro_volume_altcoin_log_reported_min_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).min())
temp_df['macro_volume_altcoin_log_reported_max_daily_tm28'] = temp_df.altcoin_volume_24h_reported_cmc.transform(lambda x: np.log(x).rolling(28).max())
temp_df['macro_volume_btc_sum_daily_tm7'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(7).sum())
temp_df['macro_volume_btc_avg_daily_tm7'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_volume_btc_min_daily_tm7'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_volume_btc_max_daily_tm7'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_volume_btc_sum_daily_tm28'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(28).sum())
temp_df['macro_volume_btc_avg_daily_tm28'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_volume_btc_min_daily_tm28'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_volume_btc_max_daily_tm28'] = temp_df.macro_volume_btc_t.transform(lambda x: x.rolling(28).max())
temp_df['macro_volume_btc_log_sum_daily_tm7'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(7).sum())
temp_df['macro_volume_btc_log_avg_daily_tm7'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_volume_btc_log_min_daily_tm7'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_volume_btc_log_max_daily_tm7'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_volume_btc_log_sum_daily_tm28'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(28).sum())
temp_df['macro_volume_btc_log_avg_daily_tm28'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_volume_btc_log_min_daily_tm28'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_volume_btc_log_max_daily_tm28'] = temp_df.macro_volume_btc_log_t.transform(lambda x: x.rolling(28).max())
temp_df = temp_df.drop(['total_volume_24h_cmc', 'altcoin_volume_24h_cmc',
                        'total_volume_24h_reported_cmc', 'altcoin_volume_24h_reported_cmc'], axis=1)
df = df.drop(['total_volume_24h_cmc', 'altcoin_volume_24h_cmc',
              'total_volume_24h_reported_cmc', 'altcoin_volume_24h_reported_cmc'], axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Market cap variables
temp_df = df[df.asset == 'bitcoin'][['date', 'total_market_cap_cmc', 'altcoin_market_cap_cmc', 
                                     'covar_mcap_t', 'covar_mcap_log_t']]
temp_df = temp_df.rename(columns={'covar_mcap_t': 'macro_mcap_btc_t',
                                  'covar_mcap_log_t': 'macro_mcap_btc_log_t'})
temp_df['macro_mcap_t']             = temp_df.total_market_cap_cmc.values
temp_df['macro_mcap_altcoin_t']     = temp_df.altcoin_market_cap_cmc.values
temp_df['macro_mcap_log_t']         = np.log(temp_df.total_market_cap_cmc.values)
temp_df['macro_mcap_altcoin_log_t'] = np.log(temp_df.altcoin_market_cap_cmc.values)
assert(0==temp_df[temp_df.total_market_cap_cmc == 0].shape[0])
assert(0==temp_df[temp_df.altcoin_market_cap_cmc == 0].shape[0])
temp_df['macro_mcap_sum_daily_tm7']      = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(7).sum())
temp_df['macro_mcap_avg_daily_tm7']      = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(7).mean())
temp_df['macro_mcap_min_daily_tm7']      = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(7).min())
temp_df['macro_mcap_max_daily_tm7']      = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(7).max())
temp_df['macro_mcap_log_sum_daily_tm7']  = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).sum())
temp_df['macro_mcap_log_avg_daily_tm7']  = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).mean())
temp_df['macro_mcap_log_min_daily_tm7']  = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).min())
temp_df['macro_mcap_log_max_daily_tm7']  = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).max())
temp_df['macro_mcap_ret_t']              = temp_df.total_market_cap_cmc.transform(lambda x: x.pct_change(periods=7))
temp_df['macro_mcap_sum_daily_tm28']     = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(28).sum())
temp_df['macro_mcap_avg_daily_tm28']     = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(28).mean())
temp_df['macro_mcap_min_daily_tm28']     = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(28).min())
temp_df['macro_mcap_max_daily_tm28']     = temp_df.total_market_cap_cmc.transform(lambda x: x.rolling(28).max())
temp_df['macro_mcap_log_sum_daily_tm28'] = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).sum())
temp_df['macro_mcap_log_avg_daily_tm28'] = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).mean())
temp_df['macro_mcap_log_min_daily_tm28'] = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).min())
temp_df['macro_mcap_log_max_daily_tm28'] = temp_df.total_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).max())
temp_df['macro_mcap_ret_tm28']           = temp_df.total_market_cap_cmc.transform(lambda x: x.pct_change(periods=28))
temp_df['macro_mcap_altcoin_sum_daily_tm7']      = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(7).sum())
temp_df['macro_mcap_altcoin_avg_daily_tm7']      = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(7).mean())
temp_df['macro_mcap_altcoin_min_daily_tm7']      = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(7).min())
temp_df['macro_mcap_altcoin_max_daily_tm7']      = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(7).max())
temp_df['macro_mcap_altcoin_log_sum_daily_tm7']  = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).sum())
temp_df['macro_mcap_altcoin_log_avg_daily_tm7']  = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).mean())
temp_df['macro_mcap_altcoin_log_min_daily_tm7']  = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).min())
temp_df['macro_mcap_altcoin_log_max_daily_tm7']  = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(7).max())
temp_df['macro_mcap_altcoin_ret_t']              = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.pct_change(periods=7))
temp_df['macro_mcap_altcoin_sum_daily_tm28']     = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(28).sum())
temp_df['macro_mcap_altcoin_avg_daily_tm28']     = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(28).mean())
temp_df['macro_mcap_altcoin_min_daily_tm28']     = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(28).min())
temp_df['macro_mcap_altcoin_max_daily_tm28']     = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.rolling(28).max())
temp_df['macro_mcap_altcoin_log_sum_daily_tm28'] = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).sum())
temp_df['macro_mcap_altcoin_log_avg_daily_tm28'] = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).mean())
temp_df['macro_mcap_altcoin_log_min_daily_tm28'] = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).min())
temp_df['macro_mcap_altcoin_log_max_daily_tm28'] = temp_df.altcoin_market_cap_cmc.transform(lambda x: np.log(x).rolling(28).max())
temp_df['macro_mcap_altcoin_ret_tm28']           = temp_df.altcoin_market_cap_cmc.transform(lambda x: x.pct_change(periods=28))

# Form market cap variables from ATH and cycle low
def idxMax(func_df, col):
    idxmax = [func_df.date.values[0]]
    maxvalue = func_df[col].values[0]
    for i in range(1, func_df.shape[0]):
        if func_df[col].values[i] > maxvalue:
            idxmax.append(func_df.date.values[i])
            maxvalue = func_df[col].values[i]
        else:
            idxmax.append(idxmax[i-1])
    return idxmax
def cycleLowMcap(func_df, mcap_col, cummax_date_col):
    cycle_low_mcap_list = [func_df[mcap_col].values[0]]
    for i in range(1, func_df.shape[0]):
        start_date = func_df[cummax_date_col].values[i]
        end_date   = func_df.date.values[i]
        new_cycle_low = np.min(func_df[(func_df.date >= start_date) & 
                                       (func_df.date <= end_date)][mcap_col].values)
        cycle_low_mcap_list.append(new_cycle_low) 
    return cycle_low_mcap_list
temp_df['temp_cummax'] = temp_df.total_market_cap_cmc.cummax()
temp_df['temp_cummax_date'] = idxMax(temp_df[['date', 'total_market_cap_cmc']],
                                              'total_market_cap_cmc')
temp_df['macro_mcap_ret_ath_t'] = (temp_df.macro_mcap_t - temp_df.temp_cummax)/temp_df.temp_cummax
temp_df['temp_altcoin_cummax'] = temp_df.altcoin_market_cap_cmc.cummax()
temp_df['temp_altcoin_cummax_date'] = idxMax(temp_df[['date', 'altcoin_market_cap_cmc']], 
                                                      'altcoin_market_cap_cmc')
temp_df['macro_mcap_altcoin_ret_ath_t'] = (temp_df.macro_mcap_altcoin_t - temp_df.temp_altcoin_cummax)/temp_df.temp_altcoin_cummax
temp_df['temp_cyclelow_mcap'] = cycleLowMcap(temp_df[['date', 'total_market_cap_cmc', 'temp_cummax_date']],
                                                     'total_market_cap_cmc', 'temp_cummax_date')
temp_df['macro_mcap_ret_low_t'] = (temp_df.macro_mcap_t - temp_df.temp_cyclelow_mcap)/temp_df.temp_cyclelow_mcap
temp_df['temp_cyclelow_altcoin_mcap'] = cycleLowMcap(temp_df[['date', 'altcoin_market_cap_cmc', 
                                                              'temp_altcoin_cummax_date']],
                                                              'altcoin_market_cap_cmc', 'temp_altcoin_cummax_date')
temp_df['macro_mcap_altcoin_ret_low_t'] = (temp_df.macro_mcap_altcoin_t - 
                                           temp_df.temp_cyclelow_altcoin_mcap)/temp_df.temp_cyclelow_altcoin_mcap
temp_df = temp_df.drop(['temp_cummax', 'temp_cummax_date',
                        'temp_altcoin_cummax', 'temp_altcoin_cummax_date',
                        'temp_cyclelow_mcap', 'temp_cyclelow_altcoin_mcap'], axis=1)
temp_df = temp_df.drop(['total_market_cap_cmc', 'altcoin_market_cap_cmc'], axis=1)
df = df.drop(['total_market_cap_cmc', 'altcoin_market_cap_cmc'], axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Set up coinmetrics columns
df = df.rename(columns = {'btc_fee_mean_usd_cm':  'macro_btc_fee_avg_t',
                          'btc_fee_med_usd_cm':   'macro_btc_fee_med_t',
                          'btc_fee_total_usd_cm': 'macro_btc_fee_sum_t',
                          'eth_fee_mean_usd_cm':  'macro_eth_fee_avg_t',
                          'eth_fee_med_usd_cm':   'macro_eth_fee_med_t',
                          'eth_fee_total_usd_cm': 'macro_eth_fee_sum_t',
                          'grayscale_defi_assets_in_usd_cm': 'macro_grayscale_defi_assets_t',
                          'grayscale_eth_assets_in_usd_cm': 'macro_grayscale_eth_assets_t',
                          'grayscale_btc_assets_in_usd_cm': 'macro_grayscale_btc_assets_t'})
df = df.drop('grayscale_gbtc_assets_in_usd_cm', axis=1)
        
# Clean cmc macro columns
df = df.rename(columns = {'active_cryptocurrencies_cmc': 'macro_active_cryptos_t', 
                          'active_exchanges_cmc': 'macro_active_ex_t',
                          'active_market_pairs_cmc': 'macro_active_ex_pairs_t'})
temp_df = df[['date', 'macro_active_cryptos_t',  
              'macro_active_ex_t', 'macro_active_ex_pairs_t']]
temp_df = temp_df.drop_duplicates(subset=['date']).reset_index(drop=True)
temp_df['macro_active_cryptos_pct_chng_tm7']   = temp_df['macro_active_cryptos_t'].pct_change(periods=7)
temp_df['macro_active_cryptos_pct_chng_tm28']  = temp_df['macro_active_cryptos_t'].pct_change(periods=28)
temp_df['macro_active_ex_pct_chng_tm7']        = temp_df['macro_active_ex_t'].pct_change(periods=7)
temp_df['macro_active_ex_pct_chng_tm28']       = temp_df['macro_active_ex_t'].pct_change(periods=28)
temp_df['macro_active_ex_pairs_pct_chng_tm7']  = temp_df['macro_active_ex_pairs_t'].pct_change(periods=7)
temp_df['macro_active_ex_pairs_pct_chng_tm28'] = temp_df['macro_active_ex_pairs_t'].pct_change(periods=28)
temp_df = temp_df.drop(['macro_active_cryptos_t', 
                        'macro_active_ex_t',
                        'macro_active_ex_pairs_t'], axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Clean and put on the fed columns
fed_cols = list(df.filter(regex='_fed', axis=1).columns.values)
temp_df = df[['date']+fed_cols]
temp_df = temp_df.drop_duplicates(subset=['date']).reset_index(drop=True)
for col in fed_cols:
    col_name_core = col.lower()
    temp_df = temp_df.rename(columns={col: 'macro_'+col_name_core+'_t'})
    temp_df['macro_'+col_name_core+'_pct_chng_tm7'] = temp_df['macro_'+col_name_core+'_t'].pct_change(periods=7)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df['macro_'+col_name_core+'_t'].isnull(), 'macro_'+col_name_core+'_pct_chng_tm7'] = 0
    temp_df.loc[(temp_df['macro_'+col_name_core+'_t']==0)&(temp_df['macro_'+col_name_core+'_t'].shift(1)==0), 'macro_'+col_name_core+'_pct_chng_tm7'] = 0
    temp_df['macro_'+col_name_core+'_pct_chng_tm28'] = temp_df['macro_'+col_name_core+'_t'].pct_change(periods=28)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df['macro_'+col_name_core+'_t'].isnull(), 'macro_'+col_name_core+'_pct_chng_tm28'] = 0
    temp_df.loc[(temp_df['macro_'+col_name_core+'_t']==0)&(temp_df['macro_'+col_name_core+'_t'].shift(1)==0), 'macro_'+col_name_core+'_pct_chng_tm28'] = 0
df = df.drop(fed_cols, axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Form cmc exchange macro cols
ex_cmc_cols = ['exchange_volume_binance_us_cmc', 'exchange_volume_coinbase_cmc',
               'exchange_volume_gemini_cmc', 'exchange_volume_kraken_cmc',
               'exchange_volume_kucoin_cmc', 'exchange_pairs_binance_us_cmc',
               'exchange_pairs_coinbase_cmc', 'exchange_pairs_gemini_cmc',
               'exchange_pairs_kraken_cmc', 'exchange_pairs_kucoin_cmc']
ex_cmc_col_name_core = ['macro_ex_volume_binanceus', 'macro_ex_volume_coinbase',
                        'macro_ex_volume_gemini', 'macro_ex_volume_kraken',
                        'macro_ex_volume_kucoin', 'macro_ex_pairs_binance_us',
                        'macro_ex_pairs_coinbase', 'macro_ex_pairs_gemini',
                        'macro_ex_pairs_kraken', 'macro_ex_pairs_kucoin']
temp_df = df[['date']+ex_cmc_cols]
temp_df = temp_df.drop_duplicates(subset=['date']).reset_index(drop=True)
for col, new_col_core in zip(ex_cmc_cols, ex_cmc_col_name_core):
    temp_df = temp_df.rename(columns={col: new_col_core+'_t'})
    temp_df[new_col_core+'_sum_daily_tm7'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(7).sum())
    temp_df[new_col_core+'_avg_daily_tm7'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(7).mean())
    temp_df[new_col_core+'_min_daily_tm7'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(7).min())
    temp_df[new_col_core+'_max_daily_tm7'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(7).max())
    temp_df[new_col_core+'_pct_chng_tm7'] = temp_df[new_col_core+'_t'].pct_change(periods=7)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col_core+'_t'].isnull(), new_col_core+'_pct_chng_tm7'] = 0
    temp_df.loc[(temp_df[new_col_core+'_t']==0)&
                (temp_df[new_col_core+'_t'].shift(1)==0), new_col_core+'_pct_chng_tm7'] = 0
    temp_df[new_col_core+'_sum_daily_tm28'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(28).sum())
    temp_df[new_col_core+'_avg_daily_tm28'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(28).mean())
    temp_df[new_col_core+'_min_daily_tm28'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(28).min())
    temp_df[new_col_core+'_max_daily_tm28'] = temp_df[new_col_core+'_t'].transform(lambda x: x.rolling(28).max())
    temp_df[new_col_core+'_pct_chng_tm28'] = temp_df[new_col_core+'_t'].pct_change(periods=28)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col_core+'_t'].isnull(), new_col_core+'_pct_chng_tm28'] = 0
    temp_df.loc[(temp_df[new_col_core+'_t']==0)&
                (temp_df[new_col_core+'_t'].shift(1)==0), new_col_core+'_pct_chng_tm28'] = 0
df = df.drop(ex_cmc_cols, axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Form btc and eth dominance and macro returns
temp_df = df[df.asset == 'bitcoin'][['date', 'covar_r_t', 'covar_r_tm28', 'covar_r_daily_t']]
temp_df = temp_df.rename(columns={'covar_r_t': 'macro_btc_ret_t',
                                  'covar_r_tm28': 'macro_btc_ret_tm28'})
temp_df = temp_df[~temp_df.covar_r_daily_t.isnull()]
temp_df['macro_btc_ret_avg_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).sum())
temp_df['macro_btc_ret_min_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_btc_ret_max_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_btc_ret_std_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_btc_ret_avg_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).sum())
temp_df['macro_btc_ret_min_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_btc_ret_max_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_btc_ret_std_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).max())
temp_df = temp_df[~temp_df.macro_btc_ret_tm28.isnull()]
temp_df = temp_df.drop('covar_r_daily_t', axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
temp_df = df[df.asset == 'ethereum'][['date', 'covar_r_t', 'covar_r_tm28', 'covar_r_daily_t']]
temp_df = temp_df.rename(columns={'covar_r_t': 'macro_eth_ret_t',
                                  'covar_r_tm28': 'macro_eth_ret_tm28'})
temp_df = temp_df[~temp_df.covar_r_daily_t.isnull()]
temp_df['macro_eth_ret_avg_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).sum())
temp_df['macro_eth_ret_min_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_eth_ret_max_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_eth_ret_std_daily_tm7'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_eth_ret_avg_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).sum())
temp_df['macro_eth_ret_min_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_eth_ret_max_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_eth_ret_std_daily_tm28'] = temp_df.covar_r_daily_t.transform(lambda x: x.rolling(28).max())
temp_df = temp_df[~temp_df.macro_eth_ret_tm28.isnull()]
temp_df = temp_df.drop('covar_r_daily_t', axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
temp_df = df[['date','btc_dominance_cmc']]
temp_df = temp_df.drop_duplicates(subset=['date']).reset_index(drop=True)
temp_df = temp_df.rename(columns={'btc_dominance_cmc': 'macro_btc_dom_t'})
temp_df['macro_btc_dom_avg_daily_tm7']  = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_btc_dom_min_daily_tm7']  = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_btc_dom_max_daily_tm7']  = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_btc_dom_std_daily_tm7']  = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(7).std())
temp_df['macro_btc_dom_pct_chng_tm7']   = temp_df.macro_btc_dom_t.pct_change(periods=7)
temp_df['macro_btc_dom_avg_daily_tm28'] = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_btc_dom_min_daily_tm28'] = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_btc_dom_max_daily_tm28'] = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(28).max())
temp_df['macro_btc_dom_std_daily_tm28'] = temp_df.macro_btc_dom_t.transform(lambda x: x.rolling(28).std())
temp_df['macro_btc_dom_pct_chng_tm28']  = temp_df.macro_btc_dom_t.pct_change(periods=28)
df = df.drop('btc_dominance_cmc', axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
temp_df = df[df.asset == 'ethereum'][['date','covar_mcap_t','macro_mcap_t']]
temp_df = temp_df.drop_duplicates(subset=['date']).reset_index(drop=True)
temp_df['macro_eth_dom_t'] = temp_df.covar_mcap_t / temp_df.macro_mcap_t
temp_df = temp_df.drop(['covar_mcap_t', 'macro_mcap_t'], axis=1)
temp_df['macro_eth_dom_avg_daily_tm7']  = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_eth_dom_min_daily_tm7']  = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_eth_dom_max_daily_tm7']  = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_eth_dom_std_daily_tm7']  = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(7).std())
temp_df['macro_eth_dom_pct_chng_tm7']   = temp_df.macro_eth_dom_t.pct_change(periods=7)
temp_df['macro_eth_dom_avg_daily_tm28'] = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_eth_dom_min_daily_tm28'] = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_eth_dom_max_daily_tm28'] = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(28).max())
temp_df['macro_eth_dom_std_daily_tm28'] = temp_df.macro_eth_dom_t.transform(lambda x: x.rolling(28).std())
temp_df['macro_eth_dom_pct_chng_tm28']  = temp_df.macro_eth_dom_t.pct_change(periods=28)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Form stablecoin metrics
temp_df = df[['date','usd_per_usdt_cmc',
              'usd_mcap_usdt_cmc', 'usd_volume_24h_usdt_cmc']]
temp_df = temp_df[~temp_df.usd_per_usdt_cmc.isnull()]
temp_df = temp_df[~temp_df.usd_mcap_usdt_cmc.isnull()]
temp_df = temp_df[~temp_df.usd_volume_24h_usdt_cmc.isnull()]
temp_df = temp_df.drop_duplicates(subset=['date']).reset_index(drop=True)
temp_df['macro_usdt_volume_t'] = temp_df.usd_volume_24h_usdt_cmc.values
temp_df['macro_usdt_mcap_t'] = temp_df.usd_mcap_usdt_cmc.values
temp_df['macro_usdt_sum_daily_dev_from_1_tm7'] = temp_df.usd_per_usdt_cmc.transform(lambda x: np.abs(x-1).rolling(7).sum())
temp_df['macro_usdt_std_daily_dev_from_1_tm7'] = temp_df.usd_per_usdt_cmc.transform(lambda x: np.abs(x-1).rolling(7).std())
temp_df['macro_usdt_macro_pct_chng_tm7'] = temp_df.macro_usdt_mcap_t.pct_change(periods=7)
temp_df['macro_usdt_volume_sum_daily_tm7'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(7).sum())
temp_df['macro_usdt_volume_avg_daily_tm7'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_usdt_volume_min_daily_tm7'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_usdt_volume_max_daily_tm7'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_usdt_volume_pct_chng_tm7']  = temp_df.macro_usdt_volume_t.pct_change(periods=7)
temp_df['macro_usdt_sum_daily_dev_from_1_tm28'] = temp_df.usd_per_usdt_cmc.transform(lambda x: np.abs(x-1).rolling(28).sum())
temp_df['macro_usdt_std_daily_dev_from_1_tm28'] = temp_df.usd_per_usdt_cmc.transform(lambda x: np.abs(x-1).rolling(28).std())
temp_df['macro_usdt_macro_pct_chng_tm28'] = temp_df.macro_usdt_mcap_t.pct_change(periods=28)
temp_df['macro_usdt_volume_sum_daily_tm28'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(28).sum())
temp_df['macro_usdt_volume_avg_daily_tm28'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_usdt_volume_min_daily_tm28'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_usdt_volume_max_daily_tm28'] = temp_df.macro_usdt_volume_t.transform(lambda x: x.rolling(28).max())
temp_df['macro_usdt_volume_pct_chng_tm28']  = temp_df.macro_usdt_volume_t.pct_change(periods=28)
temp_df = temp_df.drop(['usd_per_usdt_cmc', 'usd_mcap_usdt_cmc', 
                        'usd_volume_24h_usdt_cmc'], axis=1)
df = df.drop(['usd_per_usdt_cmc', 'usd_mcap_usdt_cmc', 'usd_volume_24h_usdt_cmc'], axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='left',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Ensure san macro cols are clean and rename them
san_macro_cols =  ['token_mcap_avg_stock_to_flow_san',
                   'token_mcap_avg_mvrv_usd_intraday_san',
                   'token_mcap_avg_mcd_collat_ratio_san',
                   'token_sum_cexes_to_dex_flow_san',
                   'token_sum_exchanges_to_defi_flow_san',
                   'token_sum_whale_to_defi_flow_san',
                   'token_sum_dex_traders_to_defi_flow_san',
                   'token_sum_whale_defi_balance_san',
                   'token_sum_traders_to_defi_flow_san',
                   'token_sum_traders_defi_balance_san',
                   'ethereum_defi_total_value_locked_usd_san',
                   'ethereum_nft_trade_volume_usd_san',
                   'ethereum_nft_trades_count_san',
                   'ethereum_nft_retail_trade_volume_usd_san',
                   'ethereum_nft_whale_trade_volume_usd_san',
                   'ethereum_nft_whale_trades_count_san',
                   'ethereum_percent_of_whale_stablecoin_total_supply_san',
                   'bitcoin_miners_to_exchanges_flow_san',
                   'ethereum_miners_to_exchanges_flow_san',
                   'ethereum_miners_exchange_balance_san',
                   'ethereum_traders_to_defi_flow_san',
                   'ethereum_traders_defi_balance_san',
                   'ethereum_mvrv_usd_intraday_san',
                   'ethereum_stock_to_flow_san']
new_san_macro_col_names = ['macro_stock_to_flow',
                           'macro_mvrv',
                           'macro_mcd_collat_ratio',
                           'macro_cex_to_dex_flow',
                           'macro_ex_to_defi_flow',
                           'macro_whale_to_defi_flow',
                           'macro_dex_to_defi_flow',
                           'macro_whale_defi_balance',
                           'macro_traders_to_defi_flow',
                           'macro_trades_defi_balance',
                           'macro_eth_defi_tvl',
                           'macro_nft_trade_volume',
                           'macro_nft_trade_count',
                           'macro_nft_retail_trade_volume',
                           'macro_nft_whale_trade_volume',
                           'macro_nft_whale_trade_count',
                           'macro_eth_prct_whale_stblcoin_total_supply',
                           'macro_btc_miner_to_ex_flow',
                           'macro_eth_miner_to_ex_flow',
                           'macro_eth_miner_ex_balance',
                           'macro_eth_trader_to_defi_flow',
                           'macro_eth_trader_defi_balance',
                           'macro_eth_mvrv',
                           'macro_eth_stock_to_flow']
temp_df = df[['date']+san_macro_cols]
temp_df = temp_df.drop_duplicates(subset=['date']).reset_index(drop=True)
for col, new_col_name_core in zip(san_macro_cols, new_san_macro_col_names):
    temp_df = temp_df.rename(columns={col: new_col_name_core+'_t'})
    temp_df[new_col_name_core+'_avg_daily_tm7'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(7).mean())
    temp_df[new_col_name_core+'_std_daily_tm7'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(7).std())
    temp_df[new_col_name_core+'_min_daily_tm7'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(7).min())
    temp_df[new_col_name_core+'_max_daily_tm7'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(7).max())
    temp_df[new_col_name_core+'_pct_chng_tm7']  = temp_df[new_col_name_core+'_t'].pct_change(periods=7)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col_name_core+'_t'].isnull(), new_col_name_core+'_pct_chng_tm7'] = 0
    temp_df.loc[(temp_df[new_col_name_core+'_t']==0)&
                (temp_df[new_col_name_core+'_t'].shift(1)==0), new_col_name_core+'_pct_chng_tm7'] = 0
    temp_df[new_col_name_core+'_avg_daily_tm28'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(28).mean())
    temp_df[new_col_name_core+'_std_daily_tm28'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(28).std())
    temp_df[new_col_name_core+'_min_daily_tm28'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(28).min())
    temp_df[new_col_name_core+'_max_daily_tm28'] = temp_df[new_col_name_core+'_t'].transform(lambda x: x.rolling(28).max())
    temp_df[new_col_name_core+'_pct_chng_tm28']  = temp_df[new_col_name_core+'_t'].pct_change(periods=28)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col_name_core+'_t'].isnull(), new_col_name_core+'_pct_chng_tm28'] = 0
    temp_df.loc[(temp_df[new_col_name_core+'_t']==0)&
                (temp_df[new_col_name_core+'_t'].shift(1)==0), new_col_name_core+'_pct_chng_tm28'] = 0
df = df.drop(san_macro_cols, axis=1)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Combine SAN dev variables into a macro variable
temp_df = df[['date', 'dev_activity_contributors_count_san',
              'github_activity_san', 'github_activity_contributors_count_san',
              'dev_activ_san']]
temp_df = temp_df.groupby('date').sum()
temp_df = pd.DataFrame(temp_df.sum(axis=1))
temp_df = temp_df.rename(columns={0:'macro_dev_activ_t'})
assert(0==temp_df[temp_df.macro_dev_activ_t.isnull()].shape[0])
temp_df = temp_df.reset_index()
temp_df['macro_dev_activ_avg_daily_tm7']  = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(7).mean())
temp_df['macro_dev_activ_std_daily_tm7']  = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(7).std())
temp_df['macro_dev_activ_min_daily_tm7']  = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(7).min())
temp_df['macro_dev_activ_max_daily_tm7']  = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(7).max())
temp_df['macro_dev_activ_pct_chng_tm7']   = temp_df.macro_dev_activ_t.pct_change(periods=7)
temp_df['macro_dev_activ_avg_daily_tm28'] = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(28).mean())
temp_df['macro_dev_activ_std_daily_tm28'] = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(28).std())
temp_df['macro_dev_activ_min_daily_tm28'] = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(28).min())
temp_df['macro_dev_activ_max_daily_tm28'] = temp_df.macro_dev_activ_t.transform(lambda x: x.rolling(28).max())
temp_df['macro_dev_activ_pct_chng_tm28']  = temp_df.macro_dev_activ_t.pct_change(periods=28)
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Create macro var out of san supply on/off exchanges
temp_df = df[['date', 'supply_on_exchanges_san',
              'supply_outside_exchanges_san']]
temp_df = temp_df.groupby('date').sum()
temp_df = temp_df.reset_index()
for col, new_col in zip(['supply_on_exchanges_san', 'supply_outside_exchanges_san'],
                        ['macro_supply_on_ex', 'macro_supply_off_ex']):
    temp_df = temp_df.rename(columns={col:new_col+'_t'})
    temp_df[new_col+'_avg_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).mean())
    temp_df[new_col+'_std_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).std())
    temp_df[new_col+'_min_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).min())
    temp_df[new_col+'_max_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).max())
    temp_df[new_col+'_pct_chng_tm7']  = temp_df[new_col+'_t'].pct_change(periods=7)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col+'_t'].isnull(), new_col+'_pct_chng_tm7'] = 0
    temp_df.loc[(temp_df[new_col+'_t']==0)&
                (temp_df[new_col+'_t'].shift(1)==0), new_col+'_pct_chng_tm7'] = 0
    temp_df[new_col+'_avg_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).mean())
    temp_df[new_col+'_std_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).std())
    temp_df[new_col+'_min_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).min())
    temp_df[new_col+'_max_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).max())
    temp_df[new_col+'_pct_chng_tm28'] = temp_df[new_col+'_t'].pct_change(periods=28)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col+'_t'].isnull(), new_col+'_pct_chng_tm28'] = 0
    temp_df.loc[(temp_df[new_col+'_t']==0)&
                (temp_df[new_col+'_t'].shift(1)==0), new_col+'_pct_chng_tm28'] = 0
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

san_sent_cols = ['sentiment_balance_total_san',
                 'sentiment_negative_total_san',
                 'sentiment_positive_total_san',
                 'social_volume_total_san']
temp_df = df[['date', 'asset']+san_sent_cols]
for col in san_sent_cols:
    temp_df[col] = temp_df.groupby('asset')[col].transform(lambda x: x/np.max(x))
temp_df = temp_df.drop('asset', axis=1)
temp_df = temp_df.groupby('date').sum()
temp_df = temp_df.reset_index()
new_san_cols = ['macro_san_sent_bal', 'macro_san_sent_neg',
                'macro_san_sent_pos', 'macro_san_social_volume']
for col, new_col in zip(san_sent_cols, new_san_cols):
    temp_df = temp_df.rename(columns={col:new_col+'_t'})
    temp_df[new_col+'_avg_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).mean())
    temp_df[new_col+'_std_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).std())
    temp_df[new_col+'_min_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).min())
    temp_df[new_col+'_max_daily_tm7']  = temp_df[new_col+'_t'].transform(lambda x: x.rolling(7).max())
    temp_df[new_col+'_pct_chng_tm7']  = temp_df[new_col+'_t'].pct_change(periods=7)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col+'_t'].isnull(), new_col+'_pct_chng_tm7'] = 0
    temp_df.loc[(temp_df[new_col+'_t']==0)&
                (temp_df[new_col+'_t'].shift(1)==0), new_col+'_pct_chng_tm7'] = 0
    temp_df[new_col+'_avg_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).mean())
    temp_df[new_col+'_std_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).std())
    temp_df[new_col+'_min_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).min())
    temp_df[new_col+'_max_daily_tm28'] = temp_df[new_col+'_t'].transform(lambda x: x.rolling(28).max())
    temp_df[new_col+'_pct_chng_tm28'] = temp_df[new_col+'_t'].pct_change(periods=28)
    temp_df.loc[(temp_df.date=='2016-1-03') & temp_df[new_col+'_t'].isnull(), new_col+'_pct_chng_tm28'] = 0
    temp_df.loc[(temp_df[new_col+'_t']==0)&
                (temp_df[new_col+'_t'].shift(1)==0), new_col+'_pct_chng_tm28'] = 0
nrows_b4 = df.shape[0]
df = df.merge(temp_df,
              on='date',
              how='inner',
              validate='many_to_one')
assert(df.shape[0]==nrows_b4)
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

# Drop macro columns with any missing in 2016 and onward
macro_cols = list(df.filter(regex='macro', axis=1).columns.values)
missing_df = df[df.date.dt.year >= 2016][macro_cols].isnull().sum()
missing_df = missing_df[missing_df > 0]
df = df.drop(list(missing_df.index.values), axis=1)

In [ ]:
# CLEAN STRICTLY CROSS SECTIONALLY VARYING VARIABLES

cs_vars = ['consensus_pow',
           'sector_dex',
           'sector_cex',
           'sector_smart_contracts',
           'sector_currency',
           'sector_defi',
           'sector_data',
           'sector_games',
           'sector_stablecoin',
           'sector_compute',
           'sector_metaverse',
           'sector_privacy',
           'sector_infra',
           'sector_nft',
           'ecosystem_avalanche',
           'ecosystem_binance',
           'ecosystem_cosmos',
           'ecosystem_eth',
           'ecosystem_polkadot',
           'ecosystem_polygon',
           'ecosystem_solana',
           'mineable_cmc',
           'pos_cmc',
           'platform_cmc']
cs_vars += list(df.filter(regex='_messari', axis=1).columns.values)

# Fix vars to be purely CS varying
for col in cs_vars:
    assert(df.shape[0]==df[col].isin([0,1]).shape[0])
    df[col] = df.groupby('asset')[col].transform(lambda x: x.max())
    assets  = np.unique(df.asset.values)
    for asset in assets:
        asset_values = df[df.asset == asset][col].values
        assert(np.all(asset_values == asset_values[0]))

# Rename and confirm not missing
for col in cs_vars:
    new_col = col.replace('_cmc', '')
    new_col = new_col.replace('_messari', '')
    new_col = 'cs_'+new_col+'_t'
    df = df.rename(columns={col: new_col})


In [1]:
# TODO SAVE THE DATA FRAME FOR DESC STAT AND UNI FACTOR WORK AT DAILY FREQ WHERE WE CUT DOWN TO JUST WHEN ASSET IS TRADABLE FOR EACH MONTH

In [ ]:
# TODO DO THE NORMALIZATION OF RHS AND THEN SAVE THE PANEL FOR 

In [ ]:
# RANK NORMALIZE THE COVAR VARS TO REPLACE WITH LIN SPACED -1 TO 1
# NOTE: This cell takes ~15 minutes.
def normalizeVector(x):
    '''
    Accepts a numpy array
    Returns a numpy array of same length with values 
        from -1 and 1, inclusive, linearly spaced.
    '''
    return np.linspace(-1, 1, num=x.shape[0])

for col in covar_cols:
    df = df.sort_values(by=['date', col])
    df['temp_col'] = df[~df[col].isnull()].groupby(['date'])[col].transform(normalizeVector)
    df.loc[df[col].isnull(), 'temp_col'] = np.nan
    assert(df[col].isnull().sum()==df['temp_col'].isnull().sum())
    df = df.drop(col, axis=1)
    df = df.rename(columns={'temp_col':col})
    assert(-1 <= np.min(df[col]))
    assert(1 >= np.max(df[col]))

df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)

In [ ]:
# MAP MACRO COLS TO -1 TO 1, LOOPING OVER THE WEEKS TO RESPECT TEMPORAL DEPENDENCE
# NOTE: this cell takes about 30 minutes
max_week_idx = np.max(df.week_idx)
for col in macro_cols:
    temp_df = df[['week_idx', col]].drop_duplicates(subset='week_idx').reset_index(drop=True)
    assert(0==temp_df[col].isnull().sum())
    temp_df.loc[temp_df.week_idx==1, 'temp_col'] = 0

    for idx in range(2,int(max_week_idx)+1):   
        col_values = temp_df[temp_df.week_idx <= idx][col].values
        idx_value  = temp_df[temp_df.week_idx == idx][col].values
        assert(len(idx_value)==1)
        idx_value  = idx_value[0]
        sorted_col_values = np.sort(col_values)
        rank = np.where(sorted_col_values == idx_value)[0][0]
        normed_values = normalizeVector(sorted_col_values)
        normed_value = normed_values[rank]
        temp_df.loc[temp_df.week_idx == idx, 'temp_col'] = normed_value

    temp_df = temp_df.drop(col, axis=1)
    temp_df = temp_df.rename(columns={'temp_col': col})
    df = df.drop(col, axis=1)
    nrows_b4 = df.shape[0]
    df = df.merge(temp_df,
                  on='week_idx',
                  how='inner',
                  validate='many_to_one')
    assert(df.shape[0]==nrows_b4)


In [ ]:
# FINAL CLEAN

# Ensure columns have the appropriate range
assert(1==np.nanmax(df.drop(['r_tplus7','asset','week_idx','date'],axis=1).values))
assert(-1==np.nanmin(df.drop(['r_tplus7','asset','week_idx','date'],axis=1).values))

# Resort columns
cols = list(df.columns.values)
first_cols = ['date', 'week_idx', 'asset', 'r_tplus7']
for col in first_cols:
    cols.remove(col)
cols.sort()
df = df[first_cols + cols]

# Resort and reindex
df = df.sort_values(by=['date', 'asset']).reset_index(drop=True)


In [ ]:
# TODO WHEN DONE, LOOK AT look at DIST OF ALL COLUMNS TO ENSURE LOOKS RIGHT
# -OVERALL, OVER TIME, BY ASSET, AND OVER TIME ASSET 

In [ ]:
# TODO SAVE THE CLEAN HOURLY AND DAILY PANELS